
# TP XOR avec Réseaux de Neurones Multi-Couches

Ce notebook implémente un réseau de neurones capable de résoudre le problème XOR.
Il inclut :
- Une classe `Layer` pour représenter chaque couche du réseau
- Une classe `Model` pour construire le réseau complet
- Les fonctions d'activation `sigmoid` et `relu` ainsi que leurs dérivées
- Une phase d'entraînement et une phase de test

---


In [1]:
import numpy as np

def sigmoid(x):
    result = 1 / (1 + np.exp(-x))
    assert np.all((result >= 0) & (result <= 1)), "Sigmoid doit être entre 0 et 1"
    return result

def sigmoid_derivative(x):
    s = sigmoid(x)
    result = s * (1 - s)
    assert np.all((result >= 0) & (result <= 0.25)), "Dérivée de la sigmoïde doit être entre 0 et 0.25"
    return result

def relu(x):
    result = np.maximum(0, x)
    assert np.all(result >= 0), "ReLU doit être >= 0"
    return result

def relu_derivative(x):
    result = (x > 0).astype(float)
    assert np.all((result == 0) | (result == 1)), "Dérivée de ReLU doit être 0 ou 1"
    return result

def linear(Z):
    result = Z
    assert result.shape == Z.shape, "Linear doit renvoyer un tableau de même forme"
    return result

def linear_derivative(Z):
    result = np.ones_like(Z)
    assert np.all(result == 1), "Dérivée de la fonction linéaire doit être 1 partout"
    return result


In [2]:
# Classe Layer
class Layer:
    def __init__(self, input_size, output_size, activation='linear', seed=None):
        np.random.seed(seed)
        self.W = np.random.randn(input_size, output_size)
        self.b = np.zeros((1, output_size))

        assert self.W.shape == (input_size, output_size), "Dimension W invalide"
        assert self.b.shape == (1, output_size), "Dimension b invalide"

        if activation == 'sigmoid':
            self.activation = sigmoid
            self.activation_derivative = sigmoid_derivative
        elif activation == 'relu':
            self.activation = relu
            self.activation_derivative = relu_derivative
        elif activation == 'linear':
            self.activation = linear
            self.activation_derivative = linear_derivative
        else:
            raise ValueError("Activation non supportée")

    def forward(self, X):
        self.input = X
        self.Z = X @ self.W + self.b
        self.A = self.activation(self.Z)

        assert self.A.shape == (X.shape[0], self.W.shape[1]), "Dimension A invalide"
        return self.A

    def backward(self, dA, lr):
        m = self.input.shape[0]
        dZ = dA * self.activation_derivative(self.Z)
        dW = self.input.T @ dZ / m
        db = np.sum(dZ, axis=0, keepdims=True) / m
        dX = dZ @ self.W.T

        assert dW.shape == self.W.shape, "Mauvaise dimension pour dW"
        assert db.shape == self.b.shape, "Mauvaise dimension pour db"

        self.W -= lr * dW
        self.b -= lr * db
        return dX

In [3]:
# Classe MLP 
class MLP:
    def __init__(self, layers, learning_rate=0.01):
        self.layers = layers
        self.lr = learning_rate

    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
        return X

    def backward(self, y_true, y_pred):
        dA = 2 * (y_pred - y_true)
        for layer in reversed(self.layers):
            dA = layer.backward(dA, self.lr)

    def train(self, X, y, epochs):
        loss = float('inf')
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = np.mean((y - y_pred) ** 2)
            if epoch % 200 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.6f}")
            self.backward(y, y_pred)

        print(f"Final Loss: {loss:.6f}")

    def predict(self, X):
        return self.forward(X)

In [4]:
# Jeu de données XOR
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# Définition du modèle avec une couche cachée de 3 neurones ReLU + couche sigmoïde
layers = [
    Layer(2, 3, activation='relu', seed=30),
    Layer(3, 4, activation='sigmoid', seed=24),
    Layer(4, 1, activation='sigmoid', seed=42)
]
model = MLP(layers, learning_rate=0.1)
model.train(X, y, epochs=10000)

# Prédictions
predictions = model.predict(X)
print("\nPrédictions après entraînement :")
for i in range(len(X)):
    print(f"Entrée : {X[i]}, Prédiction : {predictions[i][0]:.4f}, Attendu : {y[i][0]}")

Epoch 0, Loss: 0.335170
Epoch 200, Loss: 0.221277
Epoch 400, Loss: 0.180423
Epoch 600, Loss: 0.102213
Epoch 800, Loss: 0.046715
Epoch 1000, Loss: 0.025152
Epoch 1200, Loss: 0.015681
Epoch 1400, Loss: 0.010869
Epoch 1600, Loss: 0.008093
Epoch 1800, Loss: 0.006344
Epoch 2000, Loss: 0.005162
Epoch 2200, Loss: 0.004313
Epoch 2400, Loss: 0.003689
Epoch 2600, Loss: 0.003207
Epoch 2800, Loss: 0.002829
Epoch 3000, Loss: 0.002524
Epoch 3200, Loss: 0.002273
Epoch 3400, Loss: 0.002066
Epoch 3600, Loss: 0.001890
Epoch 3800, Loss: 0.001740
Epoch 4000, Loss: 0.001611
Epoch 4200, Loss: 0.001498
Epoch 4400, Loss: 0.001399
Epoch 4600, Loss: 0.001312
Epoch 4800, Loss: 0.001234
Epoch 5000, Loss: 0.001164
Epoch 5200, Loss: 0.001102
Epoch 5400, Loss: 0.001045
Epoch 5600, Loss: 0.000994
Epoch 5800, Loss: 0.000947
Epoch 6000, Loss: 0.000904
Epoch 6200, Loss: 0.000864
Epoch 6400, Loss: 0.000828
Epoch 6600, Loss: 0.000795
Epoch 6800, Loss: 0.000764
Epoch 7000, Loss: 0.000735
Epoch 7200, Loss: 0.000708
Epoch 74